In [1]:
import glob
import json
import re
import numpy as np
from processing_functions import load_JSON, sort_process_flows 

In [2]:
extract_subprocess = True
flows_extracted = []
models_skipped = []

with open('./filtered_labels.json', 'r') as f:
    data = json.load(f)
    
bpmn20_filtered_id = data.keys()
temp_training_data = {}
subprocess_names = {}
# bpmn20_filtered_id = ['594748687']
for file_num in bpmn20_filtered_id:
    pools_and_lanes = False
    file = '../../thesis_data/bpmai/models/' + file_num + '.json'
#     try:
    results = load_JSON(file, extract_subprocess = extract_subprocess)
    shapes_id = results[0]
    directly_follows = results[1]
    flows = results[2]  
    subprocess_name = results[3]

    stencils = set()
    tasks_subprocesses_id = set()
    start_events_id = set()
    end_events_id = set()
    int_events_id = set()
    gateways_count = {}
    closing_gateways_id = set()
    shapes_unwanted_id = set()
    tasks_subprocesses = ['Task', 'CollapsedSubprocess', 'Subprocess']
    gateways = ['Exclusive_Databased_Gateway', 'InclusiveGateway', 'ParallelGateway']
    shapes_unwanted = ['DataObject', 'ITSystem', 'TextAnnotation', 
                    'Association_Undirected', 'Association_Unidirectional', 'MessageFlow']

    for s in shapes_id.keys():
        stencils.add(shapes_id[s])
        if re.match('(Start.)', shapes_id[s]):
            start_events_id.add(s)
        if re.match('(End.)', shapes_id[s]):
            end_events_id.add(s)
        if re.match('(Intermediate.)', shapes_id[s]):
            int_events_id.add(s)
        if shapes_id[s] in gateways:
            gateways_count.update({s: len(directly_follows[s])})
            if len(directly_follows[s]) == 1:
                closing_gateways_id.add(s)
        if shapes_id[s] in tasks_subprocesses:
            tasks_subprocesses_id.add(s)
        if shapes_id[s] in shapes_unwanted:
            shapes_unwanted_id.add(s)

    for f in directly_follows.copy():
        if f in shapes_unwanted_id:
            directly_follows.pop(f)
        if f in directly_follows.keys() and directly_follows[f]:
            for r in directly_follows[f]:
                if r in shapes_unwanted_id:
                    directly_follows[f].remove(r)

    closing_gateways_tasks_count = {}
    for g in closing_gateways_id:
        count = 0
        for f in directly_follows.values():
            count += sum(1 if re.match(g, x) else 0 for x in f)
        closing_gateways_tasks_count[g] = count

    for t in tasks_subprocesses_id:
        count = 0
        for f in directly_follows.values():
            count += sum(1 if re.match(t, x) else 0 for x in f)
        if count > 1:
            closing_gateways_tasks_count[t] = count

    for e in end_events_id:
        count = 0
        for f in directly_follows.values():
            count += sum(1 if re.match(e, x) else 0 for x in f)
        if count > 1:
            closing_gateways_tasks_count[e] = count

    shapes_wanted = set.union(tasks_subprocesses_id, start_events_id, end_events_id, int_events_id)
    temp_closing_count = closing_gateways_tasks_count.copy()
    file_data = []
    for s in start_events_id:
        data = []
        label = []
        flow = directly_follows[s]
        result = sort_process_flows(flow, directly_follows, shapes_wanted, gateways_count, temp_closing_count)
        result.insert(0,s)
        result_copy = result.copy()    
        names = flows  

        for n, obj in enumerate(result_copy):
            if (obj in start_events_id) or (obj in int_events_id) or (obj in end_events_id):
                if obj in names:
                    res = re.findall(r'\(.*?\)', names[obj])
                    if res:
                        names[obj] = res[0][1:-1]
                    else:
                        result.remove(obj)
            if obj in gateways_count:
                result.remove(obj)

        for r in result:
            if r in names:
                data.append(names[r])
        file_data.append([x.lower() for x in data])

    # this doesnt neccessarily 
    if len(file_data):
        temp_training_data.update({file_num: file_data})
        subprocess_names.update({file_num: subprocess_name.lower()})
        
#     except:
#         # print('file skipped - error occurred')
#         print(file_num)
#         models_skipped.append(file_num)


subprocess_train_dataset = {'document': temp_training_data, 'subprocess_names': subprocess_names, 
                 'models_skipped': models_skipped}

with open('labeled_dataset_subprocess.json', 'w') as f:
    json.dump(subprocess_train_dataset, f)


In [3]:
subprocess_train_dataset

{'document': {'298551572': [['arrange invoices by priority  (high first)',
    'validate invoice',
    'mark "invoice entered correctly" checkbox',
    'set sap invoice status from "parked" to ""validated"',
    'forward invoices to the registry']],
  '988384251': [['find customer']],
  '1943637279': [['select paid invoices',
    'archive invoices per payment date',
    'archive invoices per vendor']],
  '594748687': [['required new master record',
    'complete vendor master record request form',
    'staple with invoice',
    'put in dedicated outbox']],
  '16184396': [['book funds', 'maintain transaction', 'cancel']],
  '2009977896': [['sort invoices per client ',
    'sort invoice per vendor',
    'per minute',
    'stable erntry form to invoice']],
  '990055905': [['set low priority', 'set high priority', 'stamp invoice']],
  '923737321': [['experiment with lock keeper',
    'experiment with soa security']],
  '696621953': [['moderate discussion',
    'discuss issue',
    'create 

In [4]:
subprocess_names

{'298551572': 'complete validation',
 '988384251': '',
 '1943637279': 'archive invoice(s)',
 '594748687': 'request new vendor record',
 '16184396': 'process transaction',
 '2009977896': 'invoice sorting ',
 '990055905': 'confirm urgency',
 '923737321': 'search for a phd topic',
 '696621953': 'conference session',
 '924160287': 'determine invoice priority',
 '114696846': '',
 '1666082353': '',
 '136641527': 'invoice entry process ',
 '1900005796': 'check urgency',
 '94639019': 'sort invoices',
 '1627409593': 'subprocess: assess and manage cardiovascular risks (sp4)',
 '759444129': 'check compliance',
 '1455383864': 'deal with non-compliant invoice',
 '568030606': 'store data',
 '1745058074': '',
 '440062101': 'generate vendor master record ',
 '1849808459': 'fill ief',
 '663814305': 'complete invoice',
 '77036772': '',
 '1056720200': 'collect votes',
 '231942911': 'log in',
 '1083445328': 'choose seach type',
 '2021820568': 'manage others',
 '59584027': '',
 '278446714': 'request vendor

In [5]:
with open('labeled_dataset.json', 'r') as f:
    labeled_dataset = json.load(f)
    
document = labeled_dataset['document']

In [6]:
empty_doc = [k for k, v in document.items() if v == []]
len(empty_doc)

43

In [7]:
subprocess = subprocess_train_dataset['document']

for k, v in document.items():
    if k in subprocess.keys() and v == []:
        document[k] = subprocess[k]

post_empty_doc = [k for k, v in document.items() if v == []]
len(post_empty_doc)

20

In [11]:
with open('labeled_dataset.json', 'w') as f:
    json.dump(labeled_dataset, f)

In [12]:
post_empty_doc

['257661709',
 '2103936055',
 '534963988',
 '239040048',
 '1147805498',
 '1662246858',
 '938737434',
 '959329687',
 '1219076918',
 '107168023',
 '83172456',
 '839677548',
 '107227795',
 '505448326',
 '1790363357',
 '1753657768',
 '236600171',
 '97857755',
 '804027997',
 '198605208']